In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import joblib
import random
import warnings
import os

# Suppress warnings for a cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# --- Configuration (MUST match your training setup) ---
MODEL_FILE = 'smartphone_model.h5'
SCALER_FILE = 'smartphone_scaler.joblib'
SAMPLING_INTERVAL_HOURS = 6
TIMESTEPS_PER_DAY = 24 // SAMPLING_INTERVAL_HOURS
SEQUENCE_DAYS = 14
SEQUENCE_TIMESTEPS = SEQUENCE_DAYS * TIMESTEPS_PER_DAY
N_FEATURES = 10 # Number of features in the smartphone dataset

def generate_sample_sequence(failure_type="healthy"):
    """
    Generates a sample sequence of raw smartphone data for different failure scenarios.
    """
    print(f"\nGenerating a sample sequence for scenario: '{failure_type}'...")
    raw_data = []

    for i in range(SEQUENCE_TIMESTEPS):
        # Generate baseline healthy values
        healthy_reading = {
            'battery_level': random.uniform(20, 95),
            'cpu_usage_percent': random.uniform(10, 40),
            'memory_usage_percent': random.uniform(50, 70),
            'storage_usage_percent': 60 + (i / SEQUENCE_TIMESTEPS) * 5, # Storage slowly fills
            'app_crashes': 0,
            'network_signal_strength_dbm': random.uniform(-100, -85),
            'screen_on_time_minutes': random.uniform(10, 60),
            'fast_charging_active': 0,
            'speaker_volume_percent': random.uniform(0, 100),
            'ambient_temp_c': 25.0
        }

        # Apply specific failure logic
        if failure_type == "battery_failure":
            # Battery drains unusually fast and fails to hold a charge
            healthy_reading['battery_level'] *= (1 - (i / SEQUENCE_TIMESTEPS) * 0.8) # Degrades to 20% of normal
            healthy_reading['app_crashes'] = random.choice([0, 0, 1]) # More likely to crash

        elif failure_type == "overheating_failure":
            # CPU is overworked, causing high temps and memory issues
            healthy_reading['cpu_usage_percent'] += 40 * (i / SEQUENCE_TIMESTEPS) # CPU load increases
            healthy_reading['memory_usage_percent'] += 25 * (i / SEQUENCE_TIMESTEPS) # Memory usage increases
            healthy_reading['ambient_temp_c'] += 15 * (i / SEQUENCE_TIMESTEPS) # Phone gets hot
            healthy_reading['app_crashes'] = random.choice([0, 1, 2])

        raw_data.append(healthy_reading)

    return raw_data

def run_diagnostic_analysis(model, scaler, raw_data_sequence):
    """
    Generates a diagnostic report for smartphones using feature perturbation analysis.
    """
    feature_columns = [
        'battery_level', 'cpu_usage_percent', 'memory_usage_percent',
        'storage_usage_percent', 'app_crashes', 'network_signal_strength_dbm',
        'screen_on_time_minutes', 'fast_charging_active', 'speaker_volume_percent',
        'ambient_temp_c'
    ]

    df = pd.DataFrame(raw_data_sequence)
    scaled_features = scaler.transform(df[feature_columns])
    original_sequence = np.array([scaled_features])
    base_risk_score = model.predict(original_sequence, verbose=0)[0][0]

    if base_risk_score < 0.5:
        print("--- Diagnostic Report ---")
        print("Prediction: Normal Operation")
        print(f"Risk Score: {base_risk_score:.4f}\nNo anomalies detected.")
        return

    print(f"High risk detected ({base_risk_score:.4f}). Running perturbation analysis...")

    feature_impacts = {}
    # Define "healthy" values for each feature (approximates the mean of scaled healthy data)
    healthy_values_scaled = {
        'battery_level': 0.7, 'cpu_usage_percent': 0.3, 'memory_usage_percent': 0.6,
        'storage_usage_percent': 0.5, 'app_crashes': 0.0, 'network_signal_strength_dbm': 0.8,
        'screen_on_time_minutes': 0.4, 'fast_charging_active': 0.0, 'speaker_volume_percent': 0.5,
        'ambient_temp_c': 0.5
    }

    for i, feature_name in enumerate(feature_columns):
        perturbed_sequence = original_sequence.copy()
        perturbed_sequence[:, :, i] = healthy_values_scaled[feature_name]
        new_risk_score = model.predict(perturbed_sequence, verbose=0)[0][0]
        impact = base_risk_score - new_risk_score
        feature_impacts[feature_name] = impact

    impact_series = pd.Series(feature_impacts).sort_values(ascending=False)

    cause_map = {
        "battery_level": "Severe Battery Degradation",
        "cpu_usage_percent": "Sustained High CPU Load",
        "memory_usage_percent": "Critically Low Memory / High Usage",
        "ambient_temp_c": "Overheating",
        "app_crashes": "Frequent Application Crashes",
        "network_signal_strength_dbm": "Poor Network Connectivity",
        "storage_usage_percent": "Low Storage Space",
        "screen_on_time_minutes": "Excessive Screen On Time",
        "fast_charging_active": "Charging Circuit Anomaly",
        "speaker_volume_percent": "Speaker Hardware Issue"
    }

    primary_cause_feature = impact_series.index[0]
    secondary_cause_feature = impact_series.index[1]

    print("\n" + "="*40)
    print("  Smartphone Predictive Diagnostic Report")
    print("="*40)
    print(f"Prediction: Failure Imminent")
    print(f"Risk Score: {base_risk_score:.4f}")
    print("\n--- Probable Causes (based on Perturbation Analysis) ---")
    print(f"1. Primary Cause: {cause_map.get(primary_cause_feature, 'Unknown Factor')}")
    print(f"   - Details: Neutralizing the '{primary_cause_feature}' values caused the largest drop in the risk score.")
    print(f"2. Secondary Cause: {cause_map.get(secondary_cause_feature, 'Unknown Factor')}")
    print(f"   - Details: Neutralizing the '{secondary_cause_feature}' values caused the second-largest drop in risk.")
    print("\n--- Recommendation ---")
    print("Advise user to back up their data and seek technical support.")
    print("="*40)


if __name__ == "__main__":
    try:
        model = keras.models.load_model(MODEL_FILE)
        scaler = joblib.load(SCALER_FILE)
    except Exception as e:
        print(f"Fatal Error: Could not load model or scaler. {e}")
        exit()

    # --- Test different failure scenarios ---

    # Scenario 1: A battery failure
    battery_fail_sequence = generate_sample_sequence(failure_type="battery_failure")
    run_diagnostic_analysis(model, scaler, battery_fail_sequence)

    # Scenario 2: An overheating/CPU failure
    overheating_fail_sequence = generate_sample_sequence(failure_type="overheating_failure")
    run_diagnostic_analysis(model, scaler, overheating_fail_sequence)


Generating a sample sequence for scenario: 'battery_failure'...
High risk detected (0.9454). Running perturbation analysis...

  Smartphone Predictive Diagnostic Report
Prediction: Failure Imminent
Risk Score: 0.9454

--- Probable Causes (based on Perturbation Analysis) ---
1. Primary Cause: Severe Battery Degradation
   - Details: Neutralizing the 'battery_level' values caused the largest drop in the risk score.
2. Secondary Cause: Poor Network Connectivity
   - Details: Neutralizing the 'network_signal_strength_dbm' values caused the second-largest drop in risk.

--- Recommendation ---
Advise user to back up their data and seek technical support.

Generating a sample sequence for scenario: 'overheating_failure'...
High risk detected (0.9432). Running perturbation analysis...

  Smartphone Predictive Diagnostic Report
Prediction: Failure Imminent
Risk Score: 0.9432

--- Probable Causes (based on Perturbation Analysis) ---
1. Primary Cause: Frequent Application Crashes
   - Details: N